In [1]:
# visualise s2 cells at a specifc level here
from s2sphere import CellId, Cell, LatLng
import plotly.graph_objects as go

LEVEL = 4 # choose level here to see 

def get_all_cells_at_level(level):
    cells = []
    for face in range(6):  
        root = CellId.from_face_pos_level(face, 0, 0)
        begin = root.child_begin(level)
        end = root.child_end(level)
        cell = begin
        while cell != end:
            cells.append(cell)
            cell = cell.next()
    return cells


cell_ids = get_all_cells_at_level(LEVEL)

# cooridnates
lats, lons = [], []
hover_lats, hover_lons = [], []
hover_texts = []

for cell_id in cell_ids:
    cell = Cell(cell_id)

    for i in range(4):
        vertex = LatLng.from_point(cell.get_vertex(i))
        lats.append(vertex.lat().degrees)
        lons.append(vertex.lng().degrees)
    vertex = LatLng.from_point(cell.get_vertex(0))
    lats.append(vertex.lat().degrees)
    lons.append(vertex.lng().degrees)

    lats.append(None)
    lons.append(None)

    center = LatLng.from_point(cell_id.to_point())
    hover_lats.append(center.lat().degrees)
    hover_lons.append(center.lng().degrees)
    hover_texts.append(f"ID: {cell_id.id()}<br>Token: {cell_id.to_token()}")

#  plot and add cells boarders and markers 
fig = go.Figure()

fig.add_trace(go.Scattergeo(
    lat=lats,
    lon=lons,
    mode='lines',
    line=dict(color='blue', width=0.5),
    name='S2 Cell Borders',
    showlegend=False
))

fig.add_trace(go.Scattergeo(
    lat=hover_lats,
    lon=hover_lons,
    mode='markers',
    marker=dict(size=2, color='red'),
    text=hover_texts,
    hoverinfo='text',
    name='Cell Centers'
))

# globe view
fig.update_geos(
    projection_type="orthographic",
    showland=True,
    showcountries=True,
    showcoastlines=True,
)

fig.update_layout(
    title=f"S2 Cells at Level {LEVEL} with Unique IDs",
    height=800,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()


In [3]:
# here see a hierarhical view of S@ cells 
from s2sphere import CellId, Cell, LatLng
import plotly.graph_objects as go

# example coordinates
lat, lon = 48.8566, 2.3522  # Paris, France

# convert to S2 LatLng
point = LatLng.from_degrees(lat, lon)
cell_id = CellId.from_lat_lng(point)

# levels to show
levels = list(range(0, 11))

# plot setup
fig = go.Figure()

colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple', 'brown', 'pink', 'gray', 'black']

for i, level in enumerate(levels):
    level_cell_id = cell_id.parent(level)
    cell = Cell(level_cell_id)

    lats, lons = [], []

    for j in range(4):
        vertex = LatLng.from_point(cell.get_vertex(j))
        lats.append(vertex.lat().degrees)
        lons.append(vertex.lng().degrees)

    vertex = LatLng.from_point(cell.get_vertex(0))
    lats.append(vertex.lat().degrees)
    lons.append(vertex.lng().degrees)

    fig.add_trace(go.Scattergeo(
        lat=lats,
        lon=lons,
        mode='lines',
        line=dict(color=colors[i % len(colors)], width=2),
        name=f'Level {level}'
    ))

# add marker for the point
fig.add_trace(go.Scattergeo(
    lat=[lat],
    lon=[lon],
    mode='markers+text',
    marker=dict(size=6, color='red'),
    text=["Target Point"],
    textposition='top center',
    name='Point of Interest'
))

fig.update_geos(
    projection_type="orthographic",
    showland=True,
    showcountries=True,
    showcoastlines=True,
)

fig.update_layout(
    title="Hierarchical S2 Cell Visualization (Levels 1–10)",
    height=800,
    margin=dict(l=0, r=0, t=50, b=0)
)

fig.show()
